## 📦 Bước 1: Cài Đặt và Setup

In [ ]:
# Kiểm tra GPU
!nvidia-smi

In [ ]:
# Cài đặt dependencies
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install einops tqdm matplotlib pillow
!pip install trimesh[easy] pytorch3d objaverse

In [ ]:
# Mount Google Drive để lưu model
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục lưu model
!mkdir -p /content/drive/MyDrive/WaveletTriplane

SAVE_DIR = '/content/drive/MyDrive/WaveletTriplane'
print(f"Model sẽ được lưu vào: {SAVE_DIR}")

## 📥 Bước 2: Tải Dataset Thật

**Lựa chọn dataset (chọn 1 trong 2):**

### **Option A: ShapeNet Chairs** ⭐ Recommended
- 📦 Size: ~2GB (6,778 models)
- ⏰ Thời gian tải: ~5-10 phút
- 🎯 Chất lượng: Cao, chuẩn hóa tốt
- 💡 Tốt nhất cho training nhanh

### **Option B: Objaverse**
- 📦 Size: ~800MB cho 100 objects
- ⏰ Thời gian tải: ~10-15 phút
- 🎯 Chất lượng: Rất đa dạng
- 💡 Tốt cho model tổng quát

**⚠️ Lưu ý:** Chỉ chạy 1 trong 2 cell dưới đây!

In [ ]:
# Option A: Download ShapeNet subset từ Hugging Face
import os
import gdown

DATASET_DIR = '/content/shapenet_data'
os.makedirs(DATASET_DIR, exist_ok=True)

print("📥 Downloading ShapeNet Chair dataset from Google Drive...")

# ShapeNet Chairs subset (~2GB, 6778 models)
# Link public từ các nguồn mở
url = "https://drive.google.com/uc?id=1Z8gt4HdPujBNFABYrthhau9VZW10WWYe"

try:
    # Download và extract
    gdown.download(url, f'{DATASET_DIR}/shapenet_chairs.zip', quiet=False)
    
    # Extract
    import zipfile
    print("📦 Extracting dataset...")
    with zipfile.ZipFile(f'{DATASET_DIR}/shapenet_chairs.zip', 'r') as zip_ref:
        zip_ref.extractall(DATASET_DIR)
    
    print("✓ ShapeNet dataset downloaded successfully!")
    !ls -la {DATASET_DIR}
    
except Exception as e:
    print(f"⚠️ Lỗi download: {e}")
    print("⚠️ Sử dụng dataset demo nhỏ thay thế...")
    
    # Fallback: Tải dataset nhỏ hơn từ ModelNet10
    print("\n📥 Downloading ModelNet10 (backup dataset - 48MB)...")
    !wget http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip -O {DATASET_DIR}/modelnet.zip
    !unzip -q {DATASET_DIR}/modelnet.zip -d {DATASET_DIR}
    print("✓ ModelNet10 dataset ready!")
    DATASET_DIR = f'{DATASET_DIR}/ModelNet10'

In [ ]:
# Option B: Download Objaverse samples (tự động tải từ server chính thức)
import objaverse

print("📥 Downloading Objaverse objects (official source)...")
print("⏰ Thời gian: ~10-15 phút cho 100 objects")

# Tải 100 objects chất lượng cao từ Objaverse
uids = objaverse.load_uids()[:100]  # Lấy 100 objects đầu tiên

objects = objaverse.load_objects(
    uids=uids,
    download_processes=4
)

print(f"✓ Downloaded {len(objects)} objects")
print(f"📂 Location: {objaverse._VERSIONED_PATH}")
DATASET_DIR = objaverse._VERSIONED_PATH

# List một vài objects
import os
obj_files = list(objects.values())[:5]
print(f"\n📋 Sample objects:")
for obj in obj_files:
    print(f"  - {os.path.basename(obj)}")

## 🏗️ Bước 3: Copy Model Code

In [ ]:
# Cài đặt thêm rendering tools
!pip install -q pyrender trimesh pyglet

import trimesh
import pyrender
import numpy as np
from PIL import Image
import os
from pathlib import Path

def render_mesh_multiview(mesh_path, output_dir, num_views=8, resolution=256):
    """
    Render 3D mesh thành nhiều views.
    """
    try:
        # Load mesh
        mesh = trimesh.load(mesh_path, force='mesh')
        
        # Center và normalize
        mesh.vertices -= mesh.vertices.mean(axis=0)
        mesh.vertices /= np.abs(mesh.vertices).max()
        mesh.vertices *= 0.8
        
        # Create scene
        scene = pyrender.Scene()
        mesh_node = scene.add(pyrender.Mesh.from_trimesh(mesh))
        
        # Camera
        camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
        
        # Light
        light = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0)
        scene.add(light)
        
        # Renderer
        r = pyrender.OffscreenRenderer(resolution, resolution)
        
        # Render multiple views
        views = []
        for i in range(num_views):
            angle = 2 * np.pi * i / num_views
            
            # Camera position (orbit around object)
            cam_pos = np.array([
                2.0 * np.cos(angle),
                0.5,
                2.0 * np.sin(angle)
            ])
            
            # Look at center
            camera_pose = np.eye(4)
            camera_pose[:3, 3] = cam_pos
            
            # Point camera at origin
            z = -cam_pos / np.linalg.norm(cam_pos)
            x = np.cross([0, 1, 0], z)
            x = x / np.linalg.norm(x)
            y = np.cross(z, x)
            
            camera_pose[:3, 0] = x
            camera_pose[:3, 1] = y
            camera_pose[:3, 2] = z
            
            scene.add(camera, pose=camera_pose)
            
            # Render
            color, _ = r.render(scene)
            views.append(color)
            
            scene.remove_node(list(scene.get_nodes(obj=camera))[0])
        
        r.delete()
        
        # Save views
        os.makedirs(output_dir, exist_ok=True)
        for i, view in enumerate(views):
            img = Image.fromarray(view)
            img.save(os.path.join(output_dir, f'view_{i}.png'))
        
        return True
        
    except Exception as e:
        print(f"Error rendering {mesh_path}: {e}")
        return False

# Render dataset
print("🎨 Rendering 3D models to multi-view images...")
print("⏰ Thời gian: ~1-2 phút cho 100 models\n")

RENDERED_DIR = '/content/rendered_dataset'
os.makedirs(RENDERED_DIR, exist_ok=True)

# Tìm tất cả file .obj và .glb
mesh_files = []
for ext in ['*.obj', '*.glb', '*.ply']:
    mesh_files.extend(list(Path(DATASET_DIR).rglob(ext)))

# Giới hạn số lượng để demo nhanh
max_models = min(100, len(mesh_files))
mesh_files = mesh_files[:max_models]

print(f"Found {len(mesh_files)} 3D models")
print(f"Rendering {max_models} models...\n")

from tqdm import tqdm
success_count = 0

for i, mesh_path in enumerate(tqdm(mesh_files[:max_models])):
    output_dir = os.path.join(RENDERED_DIR, f'object_{i:04d}')
    
    if render_mesh_multiview(str(mesh_path), output_dir, num_views=8):
        success_count += 1

print(f"\n✓ Rendered {success_count}/{max_models} models successfully!")
print(f"📂 Saved to: {RENDERED_DIR}")

# Update DATASET_DIR
DATASET_DIR = RENDERED_DIR

## 🎨 Bước 3.5: Render 3D Models thành Images

**Quan trọng:** Dataset 3D cần render thành multi-view images trước khi train!

In [ ]:
# Clone code từ GitHub repo
!git clone https://github.com/HoangNguyennnnnnn/WaveletTriplaneDiff.git
!cd WaveletTriplaneDiff && ls -la

# Copy file cần thiết ra thư mục chính
!cp WaveletTriplaneDiff/wavelet_triplane_diffusion.py .

print("✓ Đã clone repo thành công!")
print("✓ File wavelet_triplane_diffusion.py đã sẵn sàng!")

## 🎯 Bước 4: Tạo Dataset Loader

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import os
from pathlib import Path
import torchvision.transforms as transforms

class MultiViewDataset(Dataset):
    """
    Dataset cho 3D objects với multi-view rendering.
    Load images thật từ thư mục đã render.
    """
    
    def __init__(self, data_dir, num_views=4, image_size=256):
        self.data_dir = Path(data_dir)
        self.num_views = num_views
        self.image_size = image_size
        
        # Transform với data augmentation
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.RandomHorizontalFlip(p=0.3),  # Augmentation
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Augmentation
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
        
        self.transform_target = transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
        
        # Tìm tất cả objects
        self.objects = []
        if os.path.exists(data_dir):
            for obj_dir in Path(data_dir).iterdir():
                if obj_dir.is_dir():
                    # Kiểm tra có view_0.png không
                    if (obj_dir / 'view_0.png').exists():
                        self.objects.append(obj_dir)
        
        if len(self.objects) == 0:
            print("⚠️ CẢNH BÁO: Không tìm thấy rendered images!")
            print("⚠️ Vui lòng chạy cell render 3D models trước!")
            raise ValueError("No rendered dataset found!")
        else:
            print(f"✓ Tìm thấy {len(self.objects)} objects với multi-view images")
    
    def __len__(self):
        return len(self.objects)
    
    def __getitem__(self, idx):
        obj_dir = self.objects[idx]
        
        # Load input view (view 0)
        input_path = obj_dir / 'view_0.png'
        input_view = Image.open(input_path).convert('RGB')
        input_view = self.transform(input_view)
        
        # Load target views (views 1-4)
        target_views = []
        for i in range(1, self.num_views + 1):
            view_path = obj_dir / f'view_{i}.png'
            
            if view_path.exists():
                view = Image.open(view_path).convert('RGB')
                view = self.transform_target(view)
            else:
                # Nếu thiếu view, dùng view 0
                view = Image.open(input_path).convert('RGB')
                view = self.transform_target(view)
            
            target_views.append(view)
        
        target_views = torch.stack(target_views)
        
        return {
            'image': input_view,
            'target_views': target_views,
        }

# Test dataset
print("\n📊 Testing dataset loader...")
dataset = MultiViewDataset(DATASET_DIR)
print(f"Dataset size: {len(dataset)}")

sample = dataset[0]
print(f"Input shape: {sample['image'].shape}")
print(f"Target views shape: {sample['target_views'].shape}")

# Visualize một sample
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# Input
img = sample['image'].permute(1, 2, 0).numpy()
img = (img * 0.5 + 0.5).clip(0, 1)
axes[0].imshow(img)
axes[0].set_title('Input View')
axes[0].axis('off')

# Target views
for i in range(4):
    img = sample['target_views'][i].permute(1, 2, 0).numpy()
    img = (img * 0.5 + 0.5).clip(0, 1)
    axes[i+1].imshow(img)
    axes[i+1].set_title(f'Target {i+1}')
    axes[i+1].axis('off')

plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'dataset_sample.png'), dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Dataset loaded successfully!")

## 🏋️ Bước 5: Training Configuration

In [ ]:
# Import model
from wavelet_triplane_diffusion import WaveletTriplaneDiffusion

# Training config
CONFIG = {
    'batch_size': 8,
    'num_epochs': 50,
    'learning_rate': 1e-4,
    'triplane_channels': 32,
    'num_timesteps': 1000,
    'save_every': 5,  # Save checkpoint mỗi 5 epochs
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
}

print("Training Configuration:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

## 🚀 Bước 6: Training Loop

In [ ]:
from tqdm.notebook import tqdm
import torch.optim as optim

# Create dataloader
train_dataset = MultiViewDataset(DATASET_DIR)
train_loader = DataLoader(
    train_dataset, 
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

# Create model
device = CONFIG['device']
model = WaveletTriplaneDiffusion(triplane_channels=CONFIG['triplane_channels'])
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['num_epochs'])

# Loss tracking
train_losses = []

In [ ]:
# Training function với perceptual loss
def train_epoch(model, dataloader, optimizer, device, epoch):
    model.train()
    total_loss = 0
    total_mse = 0
    
    pbar = tqdm(dataloader, desc=f'Epoch {epoch}')
    
    for batch_idx, batch in enumerate(pbar):
        images = batch['image'].to(device)
        target_views = batch['target_views'].to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images, return_intermediate=True)
        
        # Multi-view rendering loss (so sánh với tất cả target views)
        rendered = outputs['rendered_image']
        
        # Loss với multiple target views
        losses = []
        for i in range(target_views.shape[1]):
            target = target_views[:, i]
            
            # MSE loss
            mse_loss = F.mse_loss(rendered, target)
            
            # L1 loss (robust hơn)
            l1_loss = F.l1_loss(rendered, target)
            
            # Combined loss
            view_loss = 0.7 * mse_loss + 0.3 * l1_loss
            losses.append(view_loss)
        
        # Lấy loss nhỏ nhất (best matching view)
        loss = min(losses)
        
        # Regularization: triplane smoothness
        triplane = outputs['triplane_highres']
        smooth_loss = 0.01 * (
            F.l1_loss(triplane[:, :, :, :, :-1], triplane[:, :, :, :, 1:]) +
            F.l1_loss(triplane[:, :, :, :-1, :], triplane[:, :, :, 1:, :])
        )
        
        total_train_loss = loss + smooth_loss
        
        # Backward
        total_train_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_loss += total_train_loss.item()
        total_mse += loss.item()
        
        pbar.set_postfix({
            'loss': f'{total_train_loss.item():.4f}',
            'mse': f'{loss.item():.4f}'
        })
    
    avg_loss = total_loss / len(dataloader)
    avg_mse = total_mse / len(dataloader)
    
    return {'total': avg_loss, 'mse': avg_mse}

In [ ]:
# Main training loop
print("\n" + "="*80)
print("Starting Training")
print("="*80 + "\n")

best_loss = float('inf')
train_loss_history = []
mse_loss_history = []

for epoch in range(1, CONFIG['num_epochs'] + 1):
    # Train
    losses = train_epoch(model, train_loader, optimizer, device, epoch)
    avg_loss = losses['total']
    avg_mse = losses['mse']
    
    train_loss_history.append(avg_loss)
    mse_loss_history.append(avg_mse)
    
    # Log
    print(f"\nEpoch {epoch}/{CONFIG['num_epochs']}")
    print(f"  Total Loss: {avg_loss:.4f}")
    print(f"  MSE Loss: {avg_mse:.4f}")
    print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    
    # Save best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        save_path = os.path.join(SAVE_DIR, 'wavelet_triplane_best.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': best_loss,
            'config': CONFIG,
        }, save_path)
        print(f"  ✓ Saved best model to {save_path}")
    
    # Save periodic checkpoint
    if epoch % CONFIG['save_every'] == 0:
        save_path = os.path.join(SAVE_DIR, f'wavelet_triplane_epoch_{epoch}.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
            'config': CONFIG,
        }, save_path)
        print(f"  ✓ Saved checkpoint to {save_path}")
        
        # Visualize progress
        if epoch % (CONFIG['save_every'] * 2) == 0:
            model.eval()
            with torch.no_grad():
                test_sample = train_dataset[0]
                test_img = test_sample['image'].unsqueeze(0).to(device)
                output = model(test_img, return_intermediate=True)
                
                fig, axes = plt.subplots(1, 3, figsize=(12, 4))
                
                # Input
                img = test_img[0].cpu().permute(1, 2, 0).numpy()
                img = (img * 0.5 + 0.5).clip(0, 1)
                axes[0].imshow(img)
                axes[0].set_title('Input')
                axes[0].axis('off')
                
                # Target
                target = test_sample['target_views'][0].cpu().permute(1, 2, 0).numpy()
                target = (target * 0.5 + 0.5).clip(0, 1)
                axes[1].imshow(target)
                axes[1].set_title('Target')
                axes[1].axis('off')
                
                # Rendered
                rendered = output['rendered_image'][0].cpu().permute(1, 2, 0).numpy()
                rendered = (rendered * 0.5 + 0.5).clip(0, 1)
                axes[2].imshow(rendered)
                axes[2].set_title(f'Rendered (Epoch {epoch})')
                axes[2].axis('off')
                
                plt.tight_layout()
                plt.savefig(os.path.join(SAVE_DIR, f'progress_epoch_{epoch}.png'), dpi=150)
                plt.close()
            model.train()
    
    # Step scheduler
    scheduler.step()
    
    print()

print("\n" + "="*80)
print("Training Complete!")
print(f"Best Loss: {best_loss:.4f}")
print(f"Models saved to: {SAVE_DIR}")
print("="*80)

# Save loss history
train_losses = {'total': train_loss_history, 'mse': mse_loss_history}

## 📊 Bước 7: Visualize Training

In [ ]:
import matplotlib.pyplot as plt

# Plot training loss
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Total loss
axes[0].plot(train_losses['total'], label='Total Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss (Total)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MSE loss
axes[1].plot(train_losses['mse'], label='MSE Loss', color='orange', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Training Loss (MSE)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'training_loss.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Loss curves saved to {SAVE_DIR}/training_loss.png")

## 🎨 Bước 8: Test Model

In [ ]:
# Test với một sample
model.eval()

with torch.no_grad():
    # Get a test sample
    test_sample = train_dataset[0]
    test_image = test_sample['image'].unsqueeze(0).to(device)
    
    # Forward pass
    output = model(test_image, return_intermediate=True)
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
    # Input
    img = test_image[0].cpu().permute(1, 2, 0).numpy()
    img = (img * 0.5 + 0.5).clip(0, 1)
    axes[0].imshow(img)
    axes[0].set_title('Input View')
    axes[0].axis('off')
    
    # Rendered output
    rendered = output['rendered_image'][0].cpu().permute(1, 2, 0).numpy()
    rendered = (rendered * 0.5 + 0.5).clip(0, 1)
    axes[1].imshow(rendered)
    axes[1].set_title('Rendered Output')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, 'test_output.png'), dpi=150, bbox_inches='tight')
    plt.show()
    
print("✓ Test completed!")

## 💾 Bước 9: Load Model Đã Train (Để Dùng Sau)

In [ ]:
# Function để load model đã train
def load_trained_model(checkpoint_path, device='cuda'):
    """
    Load model đã train từ checkpoint.
    
    Args:
        checkpoint_path: Đường dẫn đến file .pth
        device: 'cuda' hoặc 'cpu'
    
    Returns:
        model: Model đã load weights
        config: Training config
    """
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    # Get config
    config = checkpoint.get('config', {'triplane_channels': 32})
    
    # Create model
    model = WaveletTriplaneDiffusion(
        triplane_channels=config.get('triplane_channels', 32)
    )
    
    # Load weights
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()
    
    print(f"✓ Loaded model from epoch {checkpoint['epoch']}")
    print(f"  Loss: {checkpoint['loss']:.4f}")
    
    return model, config

# Example: Load best model
best_model_path = os.path.join(SAVE_DIR, 'wavelet_triplane_best.pth')

if os.path.exists(best_model_path):
    loaded_model, loaded_config = load_trained_model(best_model_path, device=device)
    print("\n✓ Model sẵn sàng để inference!")
else:
    print("⚠️ Chưa có model - hãy train trước!")

## 🎯 Bước 10: Inference với Model Đã Train

In [ ]:
# Function để generate 3D từ ảnh
@torch.no_grad()
def generate_3d(model, input_image, device='cuda'):
    """
    Generate 3D representation từ single image.
    
    Args:
        model: Trained model
        input_image: PIL Image hoặc tensor (1, 3, 256, 256)
        device: 'cuda' or 'cpu'
    
    Returns:
        outputs: Dict chứa triplane và rendered image
    """
    model.eval()
    
    # Preprocess if PIL Image
    if not isinstance(input_image, torch.Tensor):
        transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
        input_image = transform(input_image).unsqueeze(0)
    
    input_image = input_image.to(device)
    
    # Generate
    outputs = model(input_image, return_intermediate=True)
    
    return outputs

# Test inference
if os.path.exists(best_model_path):
    test_img = torch.randn(1, 3, 256, 256).to(device)
    result = generate_3d(loaded_model, test_img, device=device)
    
    print("Inference results:")
    print(f"  Triplane shape: {result['triplane_highres'].shape}")
    print(f"  Rendered image shape: {result['rendered_image'].shape}")
    print("\n✓ Inference successful!")

## 📥 Bước 11: Download Model về Máy (Optional)

In [ ]:
# Download model về máy local
from google.colab import files

print("📥 Downloading trained model...")
files.download(best_model_path)
print("✓ Download complete!")

## 📝 Summary

### ✅ Bạn đã hoàn thành:

1. ✅ Setup môi trường trên Colab
2. ✅ Tải dataset tự động
3. ✅ Train model với GPU miễn phí
4. ✅ Lưu model vào Google Drive
5. ✅ Test và visualize kết quả

### 📂 Files đã lưu:

```
/content/drive/MyDrive/WaveletTriplane/
├── wavelet_triplane_best.pth          # Best model
├── wavelet_triplane_epoch_*.pth       # Checkpoints
├── training_loss.png                  # Loss curve
└── test_output.png                    # Test results
```

### 🎯 Sử dụng sau này:

```python
# Load model
model, config = load_trained_model('path/to/best.pth')

# Generate 3D
result = generate_3d(model, your_image)
```

---

**🎉 Chúc mừng! Bạn đã có model riêng của mình!**